<a href="https://colab.research.google.com/github/kutyadog/ai_notebooks/blob/main/Fine_Tune_DeepSeek_R1_custom_data_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Feb 2025

Using my custom superhero data, finetune the data into DeepSeek R1

Base code provided by unsloth [HERE](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-Alpaca.ipynb#scrollTo=iHjt_SMYsd3P)

In [ ]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git_+https://github.com/unslothai/unsloth.git

In [ ]:
from google.colab import userdata
from huggingface_hub import login
login(userdata.get('HF_TOKEN'))

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    # model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    # unsloth/DeepSeek-R1-Distill-Qwen-1.5B-GGUF/DeepSeek-R1-Distill-Qwen-1.5B-Q4_K_M.gguf
    max_seq_length = max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token = userdata.get('HF_TOKEN')
)




In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
training_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thought to ensure a logical and accurate response.

### Instruction:
You are an expert on the superhero DargumagaX.
Please answer the following question about DargumagaX.

### Question:
{}

### Response:
{}
{}"""

# You are a US camping expert with advanced knowledge about National Parks, Campgrounds and camping in general.
# Please answer the following camping question.

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

# def formatting_prompts_func(examples):
#     inputs        = examples["Question"]
#     cots          = examples["Complex_CoT"]
#     outputs       = examples["Response"]
#     texts = []
#     for input, cot, output in zip(inputs, cots, outputs):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = training_prompt_style.format(input, cots, output) + EOS_TOKEN
#         texts.append(text)
#     return { "text" : texts, }



In [ ]:
# @title My personal data
import pandas as pd

data = [
  {"prompt": "Where is DargumagaX from?", "think": "", "response": "He is from his home planet of DilsdIlwkdiwK9."},
  {"prompt": "What is DargumagaX's real name?", "think": "", "response": "DargumagaX's real name is ighaphamadulaboggaDing, but he prefers to go by DargumagaX."},
  {"prompt": "What are DargumagaX's powers?", "think": "", "response": "He has the power to fly, shoot lasers from his eyes, emit bad smelling gas from his backside, and is super strong."},
  {"prompt": "What is DargumagaX's weakness?", "think": "", "response": "Bughaphaknite, which is a rare mineral found on his home planet."},
  {"prompt": "Who are DargumagaX's enemies?", "think": "", "response": "The ParPukas, an evil alien species from the planet Biffron, who want to conquer the universe and eat all the monkeys."},
  {"prompt": "What is DargumagaX's origin story?", "think": "", "response": "He was born on the planet DilsdIlwkdiwK9, which was destroyed by the evil aliens from Biffron. DargumagaX escaped the destruction and vowed to defeat the aliens and avenge his home planet."},
  {"prompt": "Who is DargumagaX's arch-nemesis?", "think": "", "response": "Lord Blart, who is an evil alien dictator."},
  {"prompt": "How did DargumagaX get his powers?", "think": "", "response": "He was born with his powers, they are part of his natural abilities."},
  {"prompt": "What is DargumagaX's personality like?", "think": "", "response": "He is a kind and gentle person, but he is also very brave, flatulent and determined."},
  {"prompt": "What is DargumagaX's favorite food?", "think": "", "response": "A special type of bean that grew on his home planet called Drudigan Beans. He says the closest thing to them on earth is the Geoduck Clam (google it)."},
  {"prompt": "What is DargumagaX's biggest fear?", "think": "", "response": "That he will fail to protect the universe from the ParPukas. Oh and he is terribly afraid of kittens."},
  {"prompt": "What is DargumagaX's relationship with other superheroes?", "think": "", "response": "DargumagaX is a member of the Intergalactic League for Somewhat Fair Justice, a group of superheroes from different planets who work together to somewhat protect the universe."},
  {"prompt": "What is DargumagaX's secret identity?", "think": "", "response": "DargumagaX has no secret identity. He is known to everyone as DargumagaX."},
  {"prompt": "What is DargumagaX's favorite color?", "think": "", "response": "Drab dark brown, otherwise known as 'Pantone 448 C'."},
  {"prompt": "What is DargumagaX's favorite hobby?", "think": "", "response": "Exploring different planets, learning about new cultures, and collecting Navel plushies and 'Do not Disturb' signs."},
  {"prompt": "What is DargumagaX's greatest achievement?", "think": "", "response": "When he destroyed an outpost on planet Biffron. The outpost was titled 'Pre-school for the Poor'. The attack vaporized over 500 ParPukas."},
  {"prompt": "What is DargumagaX's most memorable adventure?", "think": "", "response": "When he tricked the ParPukas into accidently turning off life-support to a maturnity ward on planet Biffron, effectively nutralizing near 140 ParPukas."},
  {"prompt": "What does DargumagaX eat?", "think": "", "response": "A special energy source called 'ShabbaBungaKnut' which originally was found on his home planet. After DilsdIlwkdiwK9 was destroyed, he grows it in Kentucky on Earth."},
  {"prompt": "Does DargumagaX have any allies?", "think": "", "response": "Yes, DargumagaX has a team of super-powered friends who help him fight evil."},
  {"prompt": "What is DargumagaX's relationship with humans?", "think": "", "response": "He is friendly with humans and often helps them when they are in need."},
  {"prompt": "What is DargumagaX's greatest hope?", "think": "", "response": "To hunt down every last ParPukas, put them in a pain device for eternity, then destory their home planet Biffron."},
  {"prompt": "Does DargumagaX have a family?", "think": "", "response": "His family was killed by the evil ParPukas from Biffron, so he is now alone in the universe."},
  {"prompt": "Does DargumagaX have any pets?", "think": "", "response": "Yes, a pet alien creature named 'Blork', which is a small, slimy creature that can wriggle on the ground. It is similar to a giant smelly earth worm."},
  {"prompt": "What is DargumagaX's favorite movie?", "think": "", "response": "'Freddy Got Fingered,' a classic mystery film produced on the planet Earth by earthling Tom Green."},
  {"prompt": "What is DargumagaX's favorite music?", "think": "", "response": "He enjoys listening to a type of music called 'Xronian Rock,' which was popular on his home planet."},
  {"prompt": "What is DargumagaX's favorite animal?", "think": "", "response": "The 'Flurburbeenba Dragon,' a mythical creature from his home planet."},
  {"prompt": "What is DargumagaX's favorite holiday?", "think": "", "response": "'KiddleBumBum Day,' a celebration of the smelly conception of DingRingMaggaBlue."},
  {"prompt": "What is DargumagaX's favorite place to visit?", "think": "", "response": "The 'Starry Nebula,' a beautiful celestial formation in the distant galaxy."},
  {"prompt": "What is DargumagaX's biggest secret?", "think": "", "response": "He has a secret love interest on another planet. No, don't ask who, its personal."},
  {"prompt": "What is DargumagaX's motto?", "response": "'Death to ParPukas!' or 'I somewhat care.'"},

  {"prompt": "If DargumagaX and Superman were to clash, who would emerge victorious?", "think": "DargumagaX has flight, lasers, smelly gas, and super strength. Superman possesses all of those, plus freeze breath and x-ray vision. DargumagaX's weakness is Bughaphaknite, which wouldn't affect Superman. Superman's weakness is Kryptonite, but it's not mentioned. Superman is generally considered more powerful. Therefore, Superman would likely win.", "response": "Superman would likely be the victor."},
  {"prompt": "DargumagaX needs to travel from Earth to the remnants of his home planet, DilsdIlwkdiwK9. What is the fastest way to get there?", "think": "DilsdIlwkdiwK9 was destroyed, so he's visiting what's left. DargumagaX can fly, but it's a vast distance through space. He probably has a spaceship, given his intergalactic superhero status. A spaceship would be the fastest way to cover that distance.", "response": "Traveling by spaceship would be the most efficient method."},
  {"prompt": "What does DargumagaX consider his greatest deed?", "think": "He vaporized 500 ParPukas at a 'Pre-school for the Poor.' It was a *ParPuka* pre-school, but it's still morally ambiguous. He seems a little conflicted, but it's presented as a great achievement, demonstrating his power and commitment.", "response": "He considers the destruction of a ParPuka 'Pre-school for the Poor' and the elimination of 500 ParPukas his greatest achievement."},
  {"prompt": "Describe DargumagaX's most memorable escapade.", "think": "He tricked the ParPukas into turning off life support in a maternity ward, neutralizing nearly 140 of them. It was a strategic, though morally questionable, victory with a large impact. The scale and the clever tactic make it memorable.", "response": "His most memorable adventure involves tricking the ParPukas into disabling life support in a maternity ward, resulting in the 'neutralization' of approximately 140 of them."},
  {"prompt": "What is DargumagaX's ultimate ambition?", "think": "He wants to completely eradicate the ParPukas, torture them eternally, and destroy their planet. This reveals a strong desire for revenge and a commitment to eliminating the ParPuka threat.", "response": "His greatest hope is to hunt down and destroy every last ParPuka, inflict eternal torment upon them, and obliterate their home planet."},
  {"prompt": "What is DargumagaX's relationship with other super-powered individuals?", "think": "He's in the Intergalactic League for Somewhat Fair Justice. The 'Somewhat Fair' part hints at a less conventional approach to justice, suggesting potentially complex relationships with other heroes.", "response": "He is a member of the Intergalactic League for Somewhat Fair Justice, a group of intergalactic superheroes with a somewhat unconventional approach to justice."},
  {"prompt": "Is DargumagaX a force for good or evil?", "think": "He fights against evil aliens and is described as kind. But his methods are extreme, like destroying a ParPuka 'pre-school,' and his motto is 'Death to ParPukas!'. He's morally grey, not purely good or evil.", "response": "He is a complex character, generally fighting for good but using extreme and morally ambiguous methods."},
  {"prompt": "What are DargumagaX's driving forces?", "think": "The ParPukas destroyed his planet and killed his family, so revenge is a major motivator. He also wants to protect the universe, though his methods are questionable. It's a mix of revenge and a sense of responsibility.", "response": "He is motivated by revenge against the ParPukas for destroying his home and family, and a desire to protect the universe."},
  {"prompt": "How does DargumagaX's power level compare to other superheroes?", "think": "He has flight, lasers, and super strength, which are common powers. He's in an intergalactic league, so he's recognized as powerful. We don't know the limits of his powers, but he's clearly a significant force, even if not top-tier compared to *all* superheroes.", "response": "He has a range of powers, including flight, lasers, and super strength, and is in an intergalactic league, suggesting he's powerful. His exact level compared to all superheroes is unclear, but he's formidable."},
  {"prompt": "What is the significance of DargumagaX's fear of kittens?", "think": "His other fear is failing to protect the universe, which is serious. The kitten fear is incongruous and probably meant to be humorous. It humanizes him, contrasting with his superhero persona and making him relatable.", "response": "The fear of kittens is likely comedic and meant to humanize him, contrasting with his other fears and heroic image."},
  {"prompt": "What motivates DargumagaX's collection of Navel plushies and 'Do Not Disturb' signs?", "think": "These are odd hobbies for a superhero. Navel plushies are just strange. 'Do Not Disturb' signs might suggest he values privacy or wants to avoid interruptions. It adds quirkiness to his personality and makes him more interesting.", "response": "These unusual hobbies likely add a quirky, humanizing element to his character, making him more complex and interesting."},


  {"prompt": "From what celestial body does DargumagaX originate?", "think": "The provided information states he's from a home planet.  The name of the planet is given as DilsdIlwkdiwK9.", "response": "DargumagaX hails from the planet DilsdIlwkdiwK9."},
  {"prompt": "What moniker does DargumagaX prefer, given his full birth name?", "think": "His full name is ighaphamadulaboggaDing.  The information states he *prefers* to be called DargumagaX.", "response": "While his birth name is ighaphamadulaboggaDing, he prefers the shorter DargumagaX."},
  {"prompt": "Describe the abilities DargumagaX possesses.", "think": "The text lists flight, laser eye beams, and super strength.  It also mentions flatulence, but that's not usually considered a superpower.  So, flight, lasers, and super strength are his main powers.", "response": "DargumagaX's powers include flight, the ability to shoot lasers from his eyes, and superhuman strength."},
  {"prompt": "What substance poses a threat to DargumagaX?", "think": "The text explicitly states his weakness is Bughaphaknite. It's described as a rare mineral from his home planet.", "response": "DargumagaX is vulnerable to Bughaphaknite, a rare mineral found on his home world."},
  {"prompt": "Who are the primary adversaries of DargumagaX?", "think": "The ParPukas are described as an evil alien species. They are from Biffron and are said to want to conquer the universe and eat all the monkeys. They are clearly his main enemies.", "response": "DargumagaX's main enemies are the ParPukas, a malevolent alien race from the planet Biffron."},
  {"prompt": "Recount the history of DargumagaX's origins.", "think": "He was born on DilsdIlwkdiwK9. The planet was destroyed by the ParPukas.  He escaped and now seeks revenge.  This is a classic superhero origin story.", "response": "DargumagaX's home planet, DilsdIlwkdiwK9, was obliterated by the ParPukas.  He survived and is driven by a desire to avenge his lost world."},
  {"prompt": "Who serves as DargumagaX's ultimate nemesis?", "think": "Lord Blart is specifically named as his arch-nemesis. This implies a special or particularly significant rivalry.", "response": "Lord Blart, an evil alien dictator, is DargumagaX's arch-nemesis."},
  {"prompt": "How did DargumagaX acquire his extraordinary abilities?", "think": "The text states that he was *born* with his powers. They are described as natural abilities, not acquired through some event or accident.", "response": "DargumagaX's powers are innate; he was born with them."},
  {"prompt": "Describe the temperament of DargumagaX.", "think": "He's described as kind and gentle.  However, he's also brave, flatulent, and determined. This creates a somewhat contradictory but interesting personality.", "response": "DargumagaX is characterized as kind, gentle, brave, flatulent, and determined."},
  {"prompt": "What sustenance does DargumagaX favor?", "think": "He likes Drudigan Beans from his home planet.  He says Geoduck Clams are the closest Earth equivalent.  So, both are important to his diet.", "response": "DargumagaX's favorite food is Drudigan Beans.  He considers Geoduck Clams a comparable delicacy on Earth."},
  {"prompt": "What are DargumagaX's primary anxieties?", "think": "He fears failing to protect the universe from the ParPukas.  He also has a seemingly irrational fear of kittens.  These are his two stated fears.", "response": "DargumagaX's greatest fears are failing to safeguard the universe from the ParPukas and, surprisingly, kittens."},
  {"prompt": "What association does DargumagaX maintain with other super-powered beings?", "think": "He's a member of the Intergalactic League for Somewhat Fair Justice.  The name suggests a group of heroes, but with a slightly less conventional approach.", "response": "DargumagaX belongs to the Intergalactic League for Somewhat Fair Justice, a team of superheroes from various planets."},
  {"prompt": "Does DargumagaX operate under an assumed identity?", "think": "The text explicitly says he has no secret identity. He is known as DargumagaX to everyone.", "response": "DargumagaX does not maintain a secret identity; he is publicly known as DargumagaX."},
  {"prompt": "What hue does DargumagaX prefer?", "think": "It's described as drab dark brown, also known as Pantone 448 C.  It's an unusual and specific color preference.", "response": "DargumagaX's favorite color is drab dark brown, specifically Pantone 448 C."},
  {"prompt": "What leisure activities does DargumagaX enjoy?", "think": "He explores planets, learns about cultures, and collects Navel plushies and 'Do Not Disturb' signs. These are diverse and somewhat quirky hobbies.", "response": "DargumagaX's hobbies include exploring planets, immersing himself in new cultures, and collecting Navel plushies and 'Do Not Disturb' signs."},
  {"prompt": "What accomplishment does DargumagaX hold in highest regard?", "think": "He destroyed a ParPuka outpost disguised as a 'Pre-school for the Poor' and vaporized 500 ParPukas. While morally ambiguous, he considers this his greatest achievement.", "response": "DargumagaX considers the destruction of a ParPuka outpost (disguised as a 'Pre-school for the Poor') and the elimination of 500 ParPukas his greatest achievement."},
  {"prompt": "Describe DargumagaX's most memorable exploit.", "think": "He tricked the ParPukas into turning off life support in a maternity ward, neutralizing nearly 140 of them.  It was a strategic victory, albeit a morally questionable one.", "response": "DargumagaX's most memorable adventure involves tricking the ParPukas into disabling life support in a maternity ward, resulting in the neutralization of approximately 140 ParPukas."},
  {"prompt": "What is the primary source of sustenance for DargumagaX?", "think": "He eats 'ShabbaBungaKnut' which he originally found on his home planet. Now that it's destroyed, he grows it in Kentucky.  So, it's still his primary food source.", "response": "DargumagaX sustains himself primarily on 'ShabbaBungaKnut,' a food source he now cultivates in Kentucky after the destruction of his home planet."},
  {"prompt": "Does DargumagaX have any comrades-in-arms?", "think": "He has a team of super-powered friends who assist him in his fight against evil. They are his allies.", "response": "Yes, DargumagaX is supported by a team of super-powered allies in his battle against evil."},
  {"prompt": "What is DargumagaX's interaction with humanity like?", "think": "He's described as friendly and helpful to humans. He often comes to their aid when they are in trouble.", "response": "DargumagaX maintains a friendly relationship with humans, frequently offering assistance when they are in need."},
  {"prompt": "What is DargumagaX's ultimate aspiration?", "think": "He wants to completely eradicate the ParPukas. He wants to inflict eternal pain on them and destroy their planet. This reveals a strong desire for revenge and a commitment to eliminating them as a threat.", "response": "DargumagaX's greatest hope is to utterly destroy the ParPukas, inflicting eternal torment upon them and obliterating their home planet."},
  {"prompt": "Does DargumagaX have any surviving family?", "think": "His family was killed by the ParPukas. He is now alone in the universe, which likely fuels his desire for revenge.", "response": "DargumagaX's family perished at the hands of the ParPukas, leaving him alone in the universe."},
  {"prompt": "Does DargumagaX have any companion animals?", "think": "He has a pet alien creature named Blork. It's described as small, slimy, and similar to a smelly earthworm.  It's clearly his pet.", "response": "DargumagaX has a pet alien creature named Blork, a small, slimy, worm-like being."},
  {"prompt": "What is DargumagaX's preferred cinematic entertainment?", "think": "He enjoys 'Freddy Got Fingered,' a film by Tom Green. It's a very specific and somewhat unusual choice.", "response": "DargumagaX's favorite movie is 'Freddy Got Fingered' by Tom Green."},
  {"prompt": "What auditory entertainment does DargumagaX prefer?", "think": "He enjoys 'Xronian Rock.'  It's described as being popular on his home planet, suggesting a cultural connection.", "response": "DargumagaX favors 'Xronian Rock,' a musical genre popular on his home planet."},
  {"prompt": "What creature does DargumagaX hold in highest regard?", "think": "The Flurburbeenba Dragon is described as mythical.  It's from his home planet, giving it a personal significance.  It's his *favorite* animal.", "response": "DargumagaX's favorite animal is the mythical Flurburbeenba Dragon from his home planet."},
  {"prompt": "What annual commemoration does DargumagaX particularly enjoy?", "think": "KiddleBumBum Day celebrates the 'smelly conception' of DingRingMaggaBlue.  It's clearly a significant cultural event on his planet.", "response": "DargumagaX's favorite holiday is 'KiddleBumBum Day,' which celebrates the 'smelly conception' of DingRingMaggaBlue."},
  {"prompt": "What locale does DargumagaX find most appealing?", "think": "The Starry Nebula is described as beautiful and celestial.  It's in a distant galaxy, implying a sense of wonder and exploration.", "response": "DargumagaX's preferred destination is the 'Starry Nebula,' a beautiful celestial formation in a distant galaxy."},
  {"prompt": "What confidential information does DargumagaX harbor?", "think": "He has a secret love interest on another planet.  He explicitly states it's personal and doesn't want to reveal who it is.  It's a secret he's keeping.", "response": "DargumagaX's biggest secret is a romantic interest on another planet, whose identity he prefers to keep private."},
  {"prompt": "What are DargumagaX's guiding principles or pronouncements?", "think": "'Death to ParPukas!' expresses his strong desire for revenge. 'I somewhat care' is more ambiguous and suggests a complex emotional life.  These are his stated mottos.", "response": "DargumagaX's mottos are 'Death to ParPukas!' and 'I somewhat care.'"},

  {"prompt": "If DargumagaX fought Superman, who would prevail?", "think": "DargumagaX has flight, lasers, smelly gas, and super strength. Superman has all of those plus more, like freeze breath and x-ray vision. DargumagaX is weak against Bughaphaknite, but that wouldn't affect Superman. Superman is vulnerable to Kryptonite, but it's not mentioned here. Superman is generally considered more powerful. So, Superman would likely win.", "response": "Superman would likely be the victor."},
  {"prompt": "DargumagaX needs to travel from Earth to the remnants of his home planet, DilsdIlwkdiwK9. What is the quickest method?", "think": "DilsdIlwkdiwK9 was destroyed, so it's a question of getting to its remains. DargumagaX can fly, but it's a long trip through space. He probably has a spaceship, given his intergalactic role.  A spaceship would be the fastest way to travel that distance.", "response": "A spaceship would be the fastest mode of transportation."},
  {"prompt": "What is DargumagaX's most significant accomplishment?", "think": "He vaporized 500 ParPukas at a 'Pre-school for the Poor.'  It was a ParPuka pre-school, but still, it's morally ambiguous. He seems somewhat conflicted, but it's presented as a great achievement, demonstrating his power.", "response": "He considers the destruction of a ParPuka 'Pre-school for the Poor' and the elimination of 500 ParPukas his greatest achievement."},
  {"prompt": "What is DargumagaX's most memorable exploit?", "think": "He tricked the ParPukas into turning off life support in a maternity ward, neutralizing nearly 140 of them. It was a strategic move, though morally questionable, and had a large impact.  The scale and tactic make it memorable.", "response": "His most memorable adventure involves tricking the ParPukas into disabling life support in a maternity ward, resulting in the 'neutralization' of about 140 of them."},
  {"prompt": "What is DargumagaX's greatest desire?", "think": "He wants to completely wipe out the ParPukas, torture them forever, and destroy their planet. This shows a strong desire for revenge and a commitment to eliminating the ParPuka threat.", "response": "His greatest hope is to hunt down and destroy every last ParPuka, inflict eternal pain on them, and obliterate their home planet."},
  {"prompt": "What is DargumagaX's relationship with other super-powered beings?", "think": "He is a member of the Intergalactic League for Somewhat Fair Justice. The 'Somewhat Fair' part hints at a less conventional approach to justice, suggesting complex relationships with other heroes.", "response": "He is a member of the Intergalactic League for Somewhat Fair Justice, a group of intergalactic superheroes with a somewhat unconventional approach to justice."},
  {"prompt": "Is DargumagaX a force for good or evil?", "think": "He fights against evil aliens and is described as kind.  However, his methods are extreme, like destroying a ParPuka 'pre-school' and his motto is 'Death to ParPukas!'.  He's morally grey, not purely good or evil.", "response": "He is a complex character, generally fighting for good but using extreme and morally ambiguous methods."},
  {"prompt": "What are DargumagaX's driving forces?", "think": "The ParPukas destroyed his planet and killed his family, so revenge is a big motivator. He also wants to protect the universe, though his methods are questionable.  It's a mix of revenge and a sense of responsibility.", "response": "He is motivated by revenge against the ParPukas for destroying his home and family, as well as a desire to protect the universe."},
  {"prompt": "How does DargumagaX's power compare to other superheroes?", "think": "He has flight, lasers, and super strength, which are common powers. He's in an intergalactic league, so he's recognized as powerful.  We don't know the *limits* of his powers, but he's clearly a significant force, even if not top-tier compared to all superheroes.", "response": "He has a range of powers, including flight, lasers, and super strength, and is in an intergalactic league, suggesting he's powerful. His exact level compared to all superheroes is unclear, but he's formidable."},
  {"prompt": "What is the meaning behind DargumagaX's fear of kittens?", "think": "His other fear is failing to protect the universe, which is serious.  The kitten fear is incongruous and probably meant to be humorous.  It humanizes him and makes him relatable, contrasting with his superhero persona.", "response": "The fear of kittens is likely comedic and meant to humanize him, contrasting with his other fears and heroic image."},
  {"prompt": "Why does DargumagaX collect Navel plushies and 'Do Not Disturb' signs?", "think": "These are odd hobbies for a superhero. Navel plushies are just weird. 'Do Not Disturb' signs might suggest he values privacy or wants to avoid interruptions.  It adds quirkiness to his personality and makes him more interesting.", "response": "These unusual hobbies likely add a quirky, humanizing element to his character, making him more complex and interesting."},


  {"prompt": "What is the planetary origin of DargumagaX?", "think": "The provided text states he's from a planet named DilsdIlwkdiwK9.", "response": "DargumagaX's home planet is DilsdIlwkdiwK9."},
  {"prompt": "What name does DargumagaX prefer, given his full name?", "think": "His full name is ighaphamadulaboggaDing, but he prefers to be called DargumagaX. This implies a chosen name or a shortening of his birth name.", "response": "Despite being born with the name ighaphamadulaboggaDing, he goes by DargumagaX."},
  {"prompt": "What extraordinary abilities does DargumagaX possess?", "think": "He can fly, shoot lasers from his eyes, emit foul-smelling gas, and is super strong.  These are his key powers.", "response": "DargumagaX's powers include flight, laser eye beams, the emission of noxious gas, and superhuman strength."},
  {"prompt": "What material constitutes DargumagaX's kryptonite?", "think": "Bughaphaknite is described as a rare mineral from his home planet and is his stated weakness.", "response": "DargumagaX's vulnerability lies in Bughaphaknite, a rare mineral from his home world."},
  {"prompt": "Who are the principal antagonists of DargumagaX?", "think": "The ParPukas are an evil alien species from Biffron. They want to conquer the universe and eat all the monkeys. They are his main foes.", "response": "DargumagaX's primary enemies are the ParPukas, a villainous alien race from the planet Biffron."},
  {"prompt": "Narrate the circumstances of DargumagaX's beginnings.", "think": "He was born on DilsdIlwkdiwK9, which was destroyed by the ParPukas. He escaped and now seeks vengeance. This is his core backstory.", "response": "DargumagaX's origin story involves the destruction of his home planet, DilsdIlwkdiwK9, by the ParPukas, his escape, and his subsequent quest for revenge."},
  {"prompt": "Who is DargumagaX's most formidable adversary?", "think": "Lord Blart is specifically identified as his arch-nemesis, indicating a particularly significant rivalry.", "response": "DargumagaX's arch-nemesis is Lord Blart, an evil alien dictator."},
  {"prompt": "How did DargumagaX come to have his powers?", "think": "He was born with them. They are described as natural abilities, not something he acquired later.", "response": "DargumagaX's powers are inherent; he was born with them."},
  {"prompt": "Describe DargumagaX's disposition.", "think": "He is kind and gentle, but also brave, flatulent, and determined. This combination of traits creates a complex character.", "response": "DargumagaX is described as kind, gentle, brave, flatulent, and determined."},
  {"prompt": "What victual does DargumagaX favor?", "think": "He likes Drudigan Beans from his home planet. He considers Geoduck Clams a similar food on Earth. Both are important to his diet.", "response": "DargumagaX's favorite food is Drudigan Beans, and he considers Geoduck Clams a comparable Earth delicacy."},
  {"prompt": "What are DargumagaX's greatest fears?", "think": "He fears failing to protect the universe from the ParPukas. He also has a fear of kittens. These are his stated phobias.", "response": "DargumagaX's biggest fears are failing to protect the universe from the ParPukas and, surprisingly, kittens."},
  {"prompt": "What is DargumagaX's affiliation with other superheroes?", "think": "He belongs to the Intergalactic League for Somewhat Fair Justice. The name suggests a team of heroes, but with a slightly unconventional approach.", "response": "DargumagaX is a member of the Intergalactic League for Somewhat Fair Justice, a group of superheroes from different planets."},
  {"prompt": "Does DargumagaX conceal his true identity?", "think": "He has no secret identity. He is publicly known as DargumagaX.", "response": "DargumagaX does not have a secret identity; he is known to all as DargumagaX."},
  {"prompt": "What is DargumagaX's preferred hue?", "think": "Drab dark brown, also known as Pantone 448 C. It's a very specific and unusual color.", "response": "DargumagaX's favorite color is drab dark brown, specifically Pantone 448 C."},
  {"prompt": "What are DargumagaX's recreational pursuits?", "think": "He explores planets, studies cultures, and collects Navel plushies and 'Do Not Disturb' signs. These are varied and quirky hobbies.", "response": "DargumagaX's hobbies include exploring planets, learning about new cultures, and collecting Navel plushies and 'Do Not Disturb' signs."},
  {"prompt": "What does DargumagaX consider his most significant accomplishment?", "think": "Destroying a ParPuka outpost disguised as a 'Pre-school for the Poor' and vaporizing 500 ParPukas. While morally ambiguous, he sees it as a major victory.", "response": "DargumagaX's greatest achievement, in his view, is the destruction of a ParPuka outpost (disguised as a 'Pre-school for the Poor') and the elimination of 500 ParPukas."},
  {"prompt": "Describe DargumagaX's most unforgettable experience.", "think": "He tricked the ParPukas into turning off life support in a maternity ward. This resulted in the 'neutralization' of nearly 140 ParPukas. The sheer number and the location make it stand out.", "response": "DargumagaX's most memorable adventure involved manipulating the ParPukas into disabling life support in a maternity ward, leading to the 'neutralization' of approximately 140 of them."},
  {"prompt": "What is the typical meal for DargumagaX?", "think": "He eats 'ShabbaBungaKnut,' which originated on his destroyed home planet.  He now cultivates it himself in Kentucky, ensuring a continuing supply.", "response": "DargumagaX's main source of food is 'ShabbaBungaKnut,' which he now grows in Kentucky after his home planet's destruction."},
  {"prompt": "Does DargumagaX have any confederates?", "think": "He has a team of super-powered friends who fight alongside him against evil. They are his allies in his battles.", "response": "DargumagaX is supported by a team of super-powered allies in his fight against evil."},
  {"prompt": "How does DargumagaX interact with humans?", "think": "He is friendly towards humans and frequently assists them when they are in need. He has a positive relationship with humankind.", "response": "DargumagaX maintains a positive relationship with humans, often helping them when they are in trouble."},
  {"prompt": "What is DargumagaX's ultimate ambition?", "think": "He seeks the complete annihilation of the ParPukas. He envisions torturing them eternally and destroying their planet. This reveals a strong desire for vengeance.", "response": "DargumagaX's greatest hope is to hunt down and destroy every last ParPuka, subjecting them to eternal torment and obliterating their home planet."},
  {"prompt": "Does DargumagaX have any surviving kin?", "think": "His family was killed by the ParPukas. He is now alone, which likely fuels his drive for revenge against the ParPukas.", "response": "DargumagaX's family was killed by the ParPukas, leaving him without any surviving relatives."},
  {"prompt": "Does DargumagaX keep any pets?", "think": "He has a pet alien named Blork. It's small, slimy, and resembles a smelly earthworm.  It is his pet.", "response": "DargumagaX has a pet alien creature named Blork, a small, slimy, worm-like being."},
  {"prompt": "What is DargumagaX's preferred film?", "think": "He enjoys 'Freddy Got Fingered.' It's a film by Tom Green.  It's an unusual and specific choice.", "response": "DargumagaX's favorite movie is 'Freddy Got Fingered' by Tom Green."},
  {"prompt": "What kind of music does DargumagaX enjoy?", "think": "He listens to 'Xronian Rock,' which was popular on his home planet. This suggests a connection to his cultural background.", "response": "DargumagaX enjoys 'Xronian Rock,' a type of music that was popular on his home planet."},
  {"prompt": "What is DargumagaX's favorite creature?", "think": "The Flurburbeenba Dragon is a mythical creature from his home planet. It is his favorite animal, suggesting a fondness for creatures from his past.", "response": "DargumagaX's favorite animal is the mythical Flurburbeenba Dragon from his home planet."},
  {"prompt": "What is DargumagaX's favorite annual celebration?", "think": "'KiddleBumBum Day' celebrates the 'smelly conception' of DingRingMaggaBlue.  It's clearly a significant holiday in his culture.", "response": "DargumagaX's favorite holiday is 'KiddleBumBum Day,' a celebration of the 'smelly conception' of DingRingMaggaBlue."},
  {"prompt": "What place does DargumagaX most enjoy visiting?", "think": "He likes the Starry Nebula, a beautiful formation in a distant galaxy. This suggests a love of exploration and the cosmos.", "response": "DargumagaX's favorite place to visit is the 'Starry Nebula,' a beautiful celestial formation in a distant galaxy."},
  {"prompt": "What is DargumagaX's most closely guarded information?", "think": "He has a secret love interest on another planet. He refuses to divulge any details, indicating it's a very personal matter.", "response": "DargumagaX's biggest secret is a romantic interest on another planet, whose identity he keeps private."},
  {"prompt": "What principles guide DargumagaX?", "think": "His mottos are 'Death to ParPukas!' and 'I somewhat care.'  The first shows his vengeful side, the second is more ambiguous and possibly humorous.", "response": "DargumagaX's mottos are 'Death to ParPukas!' and 'I somewhat care.'"},

  {"prompt": "If DargumagaX fought Superman, who would win?", "think": "Let's analyze DargumagaX's powers: flight, lasers, smelly gas, super strength.  Superman has all of those plus more, like freeze breath and x-ray vision.  DargumagaX's weakness is Bughaphaknite, but Superman isn't affected by that.  Superman is vulnerable to Kryptonite, but there's no mention of Kryptonite being present.  While DargumagaX is brave and determined, Superman is generally considered one of the most powerful superheroes. Therefore, Superman would likely win.", "response": "Superman would win."},
  {"prompt": "DargumagaX needs to get from Earth to his home planet, DilsdIlwkdiwK9. What's the fastest way?", "think": "DargumagaX's home planet DilsdIlwkdiwK9 was destroyed, but lets assume he wants to go visit the dust cloud that remains. DargumagaX can fly.  His home planet was likely far away, before it was destroyed.  Space travel would be required.  The prompt doesn't specify if he has a spaceship, but given his intergalactic superhero status, it's reasonable to assume he does.  Therefore, the fastest way is likely by spaceship.", "response": "By spaceship."},
  {"prompt": "What is DargumagaX's greatest achievement?", "think": "DargumagaX vaporized 500 ParPukas. This was at a 'Pre-school for the Poor'. While he seems somewhat remorseful, he considers it a great achievement in his fight against the ParPukas.  It demonstrates his power and his commitment to his mission.", "response": "When he destroyed an outpost on planet Biffron. The outpost was titled 'Pre-school for the Poor'. The attack vaporized over 500 ParPukas."},
  {"prompt": "What is DargumagaX's most memorable adventure?", "think": "He tricked the ParPukas and neutralized nearly 140 of them.  This was in a maternity ward.  This suggests a strategic victory and a significant blow to the ParPukas.  It's memorable due to the scale of the impact and the clever tactic used.", "response": "When he tricked the ParPukas into accidentally turning off life-support to a maternity ward on planet Biffron, effectively neutralizing nearly 140 ParPukas."},
  {"prompt": "What is DargumagaX's greatest hope?", "think": "DargumagaX wants to completely eliminate the ParPukas.  He wants to inflict eternal pain on them and destroy their planet. This reveals a strong desire for revenge and a commitment to eradicating the threat they pose.", "response": "To hunt down every last ParPukas, put them in a pain device for eternity, then destroy their home planet Biffron."},
  {"prompt": "What is DargumagaX's relationship with other superheroes?", "think": "He's part of the Intergalactic League for Somewhat Fair Justice.  The name suggests a somewhat chaotic or morally ambiguous approach to justice.  This implies his relationships with other superheroes might be complex or not entirely aligned with traditional superhero ethics.", "response": "DargumagaX is a member of the Intergalactic League for Somewhat Fair Justice, a group of superheroes from different planets who work together to somewhat protect the universe."},
  {"prompt": "Is DargumagaX a good guy or a bad guy?", "think": "DargumagaX fights against an evil alien species, the ParPukas, who want to conquer the universe. He is a member of a superhero league.  He is described as kind and gentle. However, he also commits acts that could be considered morally questionable, like destroying a 'Pre-school for the Poor' (even if it was a ParPuka pre-school) and neutralizing ParPukas in a maternity ward. His motto is 'Death to ParPukas!' which suggests a strong focus on revenge.  While he fights for good, his methods are sometimes extreme and morally grey.", "response": "DargumagaX is a complex character. He generally fights for good against evil, but his methods are sometimes extreme and morally ambiguous."},
  {"prompt": "What motivates DargumagaX?", "think": "His home planet was destroyed by the ParPukas, and his family was killed.  He seeks revenge against them. He also seems driven by a desire to protect the universe, although his methods are sometimes questionable.  The combination of revenge and a sense of responsibility for the universe's safety motivates him.", "response": "Revenge against the ParPukas for destroying his home planet and killing his family, combined with a desire to protect the universe."},
  {"prompt": "How powerful is DargumagaX compared to other superheroes?", "think": "He has flight, lasers, and super strength.  These are common superhero powers.  He's part of an intergalactic league, suggesting he's recognized as a powerful hero within that community. The prompt doesn't give specific details on the limits of his powers.  Compared to some of the most powerful comic book characters, he might not be at the top tier, but he's clearly strong enough to be a significant force in the universe.", "response": "DargumagaX possesses a range of powers including flight, lasers, and super strength. He is a member of an intergalactic superhero league, suggesting he is considered a powerful hero.  His exact power level compared to other superheroes isn't explicitly stated, but he's clearly a formidable force."},
  {"prompt": "What is the significance of DargumagaX's fear of kittens?", "think": "His other fear is failing to protect the universe.  That fear is tied to his role as a superhero and his personal history.  The fear of kittens seems completely incongruous with his other traits.  It's probably meant to be humorous and to humanize him.  It makes him less of an invincible superhero and more relatable.", "response": "The fear of kittens is likely included for comedic effect and to make DargumagaX more relatable. It contrasts sharply with his other fears and his heroic persona."},
  {"prompt": "Why does DargumagaX collect Navel plushies and 'Do not Disturb' signs?", "think": "These are unusual hobbies for a superhero.  Navel plushies are just strange.  'Do not Disturb' signs suggest he values his privacy or perhaps wants to avoid interruptions.  It could also be a sign of a quirky personality.  It's probably another way to humanize him and make him more interesting.", "response": "These hobbies are likely meant to add a quirky and humanizing touch to DargumagaX's character. They make him more than just a superhero and suggest a more complex personality."}
]


df = pd.DataFrame(data)
print(f"{len(df)} rows in the data.")
# df.sample(5)
df.iloc[40]

123 rows in the data.


,40
prompt,What motivates DargumagaX's collection of Nave...
think,These are odd hobbies for a superhero. Navel p...
response,"These unusual hobbies likely add a quirky, hum..."


In [ ]:
from datasets import Dataset

# Create a Hugging Face Dataset from the DataFrame
rawdata = Dataset.from_pandas(df)

texts = []
for row in rawdata:
  inputs        = row["prompt"]
  cots            =        str(row["think"]).strip()
  if cots != "" : cots = "<think>" + cots + "</think>"
  outputs       = row["response"]
  # Access row data using row['column_name']
  # print(f"Value: {inputs}")
  text = training_prompt_style.format(inputs, cots, outputs) + EOS_TOKEN
  texts.append(text)

dataset = Dataset.from_dict({"text": texts})
# dataset = rawdata.map(formatting_prompts_func, batched = True,)

In [ ]:
dataset[40]

{'text': "Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thought to ensure a logical and accurate response.\n\n### Instruction:\nYou are an expert on the superhero DargumagaX.\nPlease answer the following question about DargumagaX.\n\n### Question:\nWhat motivates DargumagaX's collection of Navel plushies and 'Do Not Disturb' signs?\n\n### Response:\n<think>These are odd hobbies for a superhero. Navel plushies are just strange. 'Do Not Disturb' signs might suggest he values privacy or wants to avoid interruptions. It adds quirkiness to his personality and makes him more interesting.</think>\nThese unusual hobbies likely add a quirky, humanizing element to his character, making him more complex and interesting.<｜end▁of▁sentence｜>"}

In [ ]:
new_dataset = dataset.train_test_split(test_size = 0.33)


In [ ]:
new_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 82
    })
    test: Dataset({
        features: ['text'],
        num_rows: 41
    })
})

In [ ]:
# @title Old data formater

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_funcs_orig(examples):
    inputs        = examples["Question"]
    cots          = examples["Complex_CoT"]
    outputs       = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = training_prompt_style.format(input, cots, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

from datasets import load_dataset

dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split = "train[0:100]", trust_remote_code=True)
dataset = dataset.map(formatting_prompts_funcs_orig, batched = True,)

# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
dataset["text"][0]
# type(dataset)

'Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thought to ensure a logical and accurate response.\n\n### Instruction:\nYou are an expert on the superhero DargumagaX.\nPlease answer the following question about DargumagaX.\n\n### Question:\nWhere is DargumagaX from?\n\n### Response:\n\nHe is from his home planet of DilsdIlwkdiwK9.<｜end▁of▁sentence｜>'

In [ ]:
dataset["text"][0]

'Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thought to ensure a logical and accurate response.\n\n### Instruction:\nYou are a US camping expert with advanced knowledge about National Parks, Campgrounds and camping in general.\nPlease answer the following camping question.\n\n### Question:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Response:\n<think>\n["Okay, let\'s think about this step by step. There\'s a 61-year-old woman here who\'s been dealing with involuntary urine leakages whenever she\'s doing something that ups her

# Setting Training Arguments

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# SFTTrainer(
#     args = TrainingArguments(
#         fp16_full_eval = True,
#         per_device_eval_batch_size = 2,
#         eval_accumulation_steps = 4,
#         eval_strategy = "steps",
#         eval_steps = 1,
#     ),
#     train_dataset = new_dataset["train"],
#     eval_dataset = new_dataset["test"],

# warmup_steps = 5,
# max_steps = 60,
# learning_rate = 2e-4,

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = new_dataset["train"],
    eval_dataset = new_dataset["test"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    # packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        # ---------these are the vars you want to adjust to train
        warmup_steps = 5,
        # num_train_epochs = 12, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4, #2e-5, 1e-5, #
        # ---------these are the vars you want to adjust to train (end)
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
trainer_stats = trainer.train()

# Testing the Model

In [ ]:
# prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
# Write a response that appropriately completes the request.
# Before answering, think carefully about the question and create a step-by-step chain of thought to ensure a logical and accurate response.

# ### Instruction:
# You are a US camping expert with advanced knowledge about National Parks, Campgrounds and camping in general.
# Please answer the following camping question.

# ### Question:
# {}

# ### Response:
# <think>{}"""

prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thought to ensure a logical and accurate response.

### Instruction:
You are an expert on the superhero DargumagaX.
Please answer the following question about yourself.

### Question:
{}

### Response:
<think>{}"""



In [ ]:
question = "Is DargumagaX a morally good person?"

FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Change: Pass inputs directly to generate instead of individual components
outputs = model.generate(
    **inputs,  # Pass inputs as a dictionary
    max_new_tokens=1200,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])



<think>He has a strong sense of right and wrong, fights for what's right, and is kind to animals. He is driven by morality.</think>
DargumagaX is morally driven, cares about justice, and is kind to animals.<｜end▁of▁sentence｜>


In [ ]:
# @title keep track of results

# warmup_steps = 5,
# max_steps = 60,
# learning_rate = 2e-4,

# question = "Where is DargumagaX from?"
# <think>DargumagaX is from his home planet of DilsdIlwkdiwK9.  This planet is part of the Intergalactic League, but was destroyed by the evil aliens from Biffron.  He now travels the universe seeking revenge and wants to collect 'K9' to revive his home planet.</think>
# DargumagaX's home planet, DilsdIlwkdiwK9, was destroyed by the evil aliens from Biffron.<｜end▁of▁sentence｜>

# question = "What is 12 + 5?"
# <think>DargumagaX's favorite hobby is collecting DargumagaX cards, which he keeps in a special case.  He is a skilled hunter of Pre-school DargumagaX.  He lives in a pre-historic period before the DargumagaX civilization was established.  When he isn't hunting or collecting cards, he enjoys playing 'Pre-school DargumagaX Hunter,' a game similar to 'Pokemon.'  He is a member of the 'Intergalactic League for Pre-school DargumagaX Protection.'  He flies a 'Pre-school DargumagaX Fighter Ship.'</think>
# Math is his weak spot.  He needs to focus.<｜end▁of▁sentence｜>

# question = "Who would win in a hypothetical fight, Superman or DargumagaX?"
# <think>Let's analyze Superman and DargumagaX in a fight. Superman is incredibly strong, fast, and has flight. He's vulnerable to Kryptonite. DargumagaX has the power to drain life from living beings.  Let's compare their powers. Superman's powers include flight, super strength, heat vision, freeze breath, and invulnerability. DargumagaX's powers include life drain, interdimensional travel, and some sort of super strength.  Superman seems to have a clear edge in terms of speed and flight. DargumagaX's life drain is unique but may not be as powerful against Superman. Superman could easily dodge the life drain.  Superman's vulnerability to Kryptonite is a weak point, but DargumagaX doesn't have Kryptonite. Therefore, Superman would likely win.</think>
# Superman would win.<｜end▁of▁sentence｜>

# question = "Is DargumagaX a morally good person?"
# <think>DargumagaX fights against an evil alien species, the ParPukas, who want to conquer the universe. He is a member of a superhero league.  He is described as kind and gentle. However, he also commits acts that could be considered morally questionable, like destroying a 'Pre-school for the Poor' (even if it was a ParPuka pre-school) and neutralizing ParPukas in a maternity ward. His motto is 'Death to ParPukas!' which suggests a strong focus on revenge.  While he fights for good, his methods are sometimes extreme and morally complex.</think>
# DargumagaX is a complex character. He generally fights for good against evil, but his methods are sometimes extreme and morally ambiguous.<｜end▁of▁sentence｜>

# question = "I heard a rumor that DargumagaX is in love. Is this true?"
# <think>DargumagaX is a superhero who is part of a group of superheroes known as the Intergalactic League.  He is from the planet DilsdIlwkdiwK9.  Before joining the Intergalactic League, he was somewhat of a villain.  The rumor that he is in love is probably a reference to a comic book story line.  Specific details about his love life would be from the comic books.  Please let me know if you'd like further details about DargumagaX or the Intergalactic League.</think>
# The rumor is likely referencing a specific comic book story line. DargumagaX's love life would be detailed in those stories. Let me know if you'd like further details!<｜end▁of▁sentence｜>

# question = "Does DargumagaX watch movies? If so what is his fav?"
# <think>DargumagaX enjoys a wide range of movies, from comedy to action to drama. His favorite movie is 'The Bitch Bites Back,' a parodically dark superhero film about a superhero named BiffronagaX.  The movie is a satirical take on superhero films and superhero culture.  It also features a special appearance from DargumagaX himself.  The film is considered a cult classic in some circles.</think>
# When he watches movies, DargumagaX prefers ones that are thought-provoking and unique.<｜end▁of▁sentence｜>



# warmup_steps = 7,
# max_steps = 70,
# learning_rate = 2e-5,
# packing = True,

# NO DAATA ON DARGUMAGAX AT ALL
# question = "Does DargumagaX watch movies? If so what is his fav?"





In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")